# Data engineer Job's data analysis


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import plotly.graph_objects as go


In [ ]:
df = pd.read_csv('../input/data-engineer-jobs/DataEngineer.csv')
df.head()

In [ ]:
df.shape

# Data cleaning


**There are some unnecessary coloumuns that we need to drop**

1. Competitors
1. Easy Apply


**Dropping Rows**


In [ ]:
df.drop(['Competitors','Easy Apply'], axis=1, inplace =True)

Replacing "-1" with np.nan because salary and revenue field contain and it doesnt make much sense there.

In [ ]:
df.replace(['-1'], [np.nan], inplace=True)
df.replace(['-1.0'], [np.nan], inplace=True)
df.replace([-1], [np.nan], inplace=True)

**Now check if there is any missing data**

In [ ]:
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("")
df.info()

**Now salary coloumn contains the Salary Estimate. We will seprate those range into two coloumns and clean those columns**
1. Max Salary
1. Min Salary

In [ ]:
new_salary_range = df['Salary Estimate'].str.rsplit('-',expand=True)
Min_Salary = new_salary_range[0]
Min_Salary = Min_Salary.str.replace('K','')
Min_Salary = Min_Salary.str.replace('$','', regex=True)

Max_Salary = new_salary_range[1]

#RegEx pattern to find (AAAA) anthing in and including the crackets

pattern =r'\(([^)]+)\)'
Max_Salary = Max_Salary.str.replace(pattern,'', regex=True)

Max_Salary = Max_Salary.str.replace('K','', regex=True)
Max_Salary = Max_Salary.str.replace('$','', regex=True)

In [ ]:
df['Minimum Salary(in grand)'] = Min_Salary
df['Maximum Salary(in grand)'] = Max_Salary
df.drop(['Salary Estimate'], axis =1,inplace=True)
#conversion of min salary object type to int64 dtypes
df['Minimum Salary(in grand)'] = pd.to_numeric(df['Minimum Salary(in grand)'])
df['Maximum Salary(in grand)'] = pd.to_numeric(df['Maximum Salary(in grand)'])

In [ ]:
#cleaning the company column
df['Company Name'] = df['Company Name'].str.replace('\n.*', ' ',regex=True)

**Making seprate city and state columns for both Location and Headquaters**

In [ ]:
Location = df['Location'].str.rsplit(',',expand=True)
Location_City = Location[0]
Location_State = Location[1]
df['Location City'] = Location_City
df['Location State'] = Location_State
df.drop('Location', axis =1, inplace= True)

Headquarters = df['Headquarters'].str.split(',',expand=True)
Headquarters_City = Headquarters[0]
Headquarters_State = Headquarters[1]
df['Headquarters City'] = Headquarters_City
df['Headquarters State'] = Headquarters_State
df.drop('Headquarters',axis =1, inplace= True)


**Now we will delete the unnecessary (Glassdoor.net) at the end of the Max Salary Range**

In [ ]:
df['Job Title'].value_counts()

* Job Title contains some extra data for departments that we dont need.


In [ ]:
department = df['Job Title'].str.split(',', expand = True)
df['Job Title'], df['Department'] = department[0],department[1]

In [ ]:
df['Department'].isnull().value_counts()

Since Department has too many empty cells. We will drop that row.

In [ ]:
df.drop('Department',axis =1,inplace =True)

In [ ]:
df['Job Title'].value_counts()

* There are some job titles are same but has different name For example . 
 1. Big Data Engineer == Data Engineer
 1. Data Engineer Lead == Data Engineer
* We will clean these

In [ ]:
#reaplce the Sr. with Senion
df['Job Title'] = df['Job Title'].str.replace('Sr.','Senior', regex=True)

In [ ]:
refine_job_title = list(df['Job Title'])
output = []
for element in refine_job_title:
#Re will find the pattern with the following expression [\w\s\S]*Data[\w\s\S]*Engineer[\w\s\S]*
    x = re.sub("[\w\s\S]*Data[\w\s\S]*Engineer[\w\s\S]*",'Data Engineer', element,flags=re.IGNORECASE)
    output.append(x)

In [ ]:
#converting that output column to data frame 
df['Job_Title']=pd.DataFrame(output)
#droping the Job title coumn
df.drop('Job Title',axis=1,inplace =True)

#if you find any other jib title it will cleaned through the same way I used above

In [ ]:
df['Job_Title'].value_counts()

In [ ]:
#We will clean the revenue field and divide the data into Min revenue and Max revenue 
df['Revenue'].value_counts()

In [ ]:
#subsituting the N/A value with none
df['Revenue'] = df['Revenue'].replace('Unknown / Non-Applicable',None)
#We define the RegEx pattern so that it will remove (USD) from end
pattern_1 = r'\(([^)]+)\)'
df['Revenue'] = df['Revenue'].str.replace(pattern_1,'', regex=True)
df['Revenue'] =df['Revenue'].str.replace('$','',regex=True)
df['Revenue'] =df['Revenue'].str.replace('+','',regex=True)

In [ ]:
df['Revenue'].value_counts()

In [ ]:
df['Revenue'] = df['Revenue'].str.replace('2 to 5 billion', '2billion to 5 billion',regex=True)
df['Revenue'] = df['Revenue'].str.replace('5 to 10 billion', '5billion to 10billion',regex=True)
df['Revenue'] = df['Revenue'].str.replace('1 to 2 billion', '1billion to 2billion',regex=True)
df['Revenue'] = df['Revenue'].str.replace('Less than 1 million', '0 to 1million',regex=True)
df['Revenue'] = df['Revenue'].str.replace('10 billion', '10billion to 20billion',regex=True) #$10 Billion+ for this max amount could be anything.
df['Revenue'] = df['Revenue'].str.replace(' billion', 'billion',regex=True)

In [ ]:
df['Revenue'].value_counts()

Replacing the Billions with '000' and millions with " "

In [ ]:
df['Revenue'] = df['Revenue'].str.replace('million', '',regex=True)
df['Revenue'] = df['Revenue'].str.replace('billion', '000',regex=True)

In [ ]:
df['Revenue'].value_counts()

In [ ]:
Revenue =df['Revenue'].str.split('to',expand =True)
df['Minimum Revenue'] = Revenue[0]
df['Maximum Revenue'] = Revenue[1]
df['Maximum Revenue'] = pd.to_numeric(df['Maximum Revenue'])
df['Minimum Revenue'] = pd.to_numeric(df['Minimum Revenue'])

Cleaning the Employee size columns

In [ ]:
df['Size'].value_counts()

In [ ]:
df['Size'] = df['Size'].str.replace('employees', '',regex=True)
df['Size'] = df['Size'].str.replace('+', '',regex=True)
df['Size'] = df['Size'].replace('Unknown', None)
df['Size'] = df['Size'].str.replace('10000', '11000',regex=True) #10000+ employess max could be any number. We take a ranadon number
df[['Min_EmployeeSize','Max_EmployeeSize']] = df['Size'].str.split('to',expand=True)
df.drop('Size',axis=1,inplace= True)

**Lets see our cleaned data**

In [ ]:
df.head()

# Data Visualisation

In [ ]:
jobs_total = df.groupby(['Job_Title'], as_index=False)['Company Name'].count().sort_values('Company Name',ascending=False).head(15)


In [ ]:
fig = px.bar(jobs_total, 
             x='Job_Title',
             y='Company Name', 
             title='Top 15 Jobs',
             labels = {"Company Name": "Total number of Jobs"},
             color='Job_Title',
             text='Company Name'
            )
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
# Displaying the graph
fig.show()

In [ ]:
jobs_company = df.groupby(['Company Name'], as_index=False)['Job_Title'].count().sort_values('Job_Title',ascending=False).head(20)

In [ ]:
fig1 = px.bar(jobs_company, x='Company Name',
              y='Job_Title', 
              title='Top 20 companies with highest number of jobs',
              labels = {"Job Title": "Total number of Jobs"},
              color ='Company Name',
              text='Job_Title'
             )
fig1.update_traces(texttemplate='%{text:.2s}', textposition='outside')
# Displaying the graph
fig1.show()

In [ ]:
max_hq = df.groupby(['Company Name'], as_index=False)['Headquarters City'].count().sort_values('Headquarters City',ascending=False).head(20)
hq_city = df["Headquarters City"].value_counts().sort_values(ascending=False).head(25).rename_axis('Headquarters City').reset_index(name='count')


In [ ]:
fig4 = px.pie(data_frame = max_hq, 
              names='Company Name',
              values='Headquarters City', 
              labels = {"Headquarters City": "No. of HQ's"},
              title = "Top 25 companies with Maximum number of Headquarters")   
fig4.show()

fig3 = px.pie(hq_city, 
              names='Headquarters City', 
              values='count',
              labels = {"count":"No. of HQ's"},
              title = "Total Number of Headquarters each city have")
  

fig3.show()
#fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")


In [ ]:
df_rev = df.groupby('Company Name', as_index=False)[['Minimum Revenue','Maximum Revenue']].mean()

In [ ]:
fig6 = px.scatter(df_rev,
                  x='Company Name',
                  y='Maximum Revenue', 
                  color='Company Name',
                  title='Maxmimum Revenue by each company',
                  labels = {'Maximum Revenue':'Revenue(Million Dollars)'}
                 )
fig6.show()
fig7 = px.scatter(df_rev,
                  x='Company Name',
                  y='Minimum Revenue', 
                  color='Company Name',
                  title='Minimum Revenue by each company',
                  labels = {'Minimum Revenue':'Revenue(Million Dollars)'}
                 )
fig7.show()


In [ ]:
df_sec = df.groupby('Sector', as_index=False)[['Minimum Revenue','Maximum Revenue']].mean()
df_sec.sort_values('Minimum Revenue',inplace=True, ascending=False)
df_sec['Average Revenue'] = df_sec[['Minimum Revenue','Maximum Revenue']].mean(axis=1)

In [ ]:
import plotly.graph_objects as go


fig8 = go.Figure()
fig8.add_trace(go.Bar(
    x=df_sec['Sector'],
    y=df_sec['Maximum Revenue'],
    text=df_sec['Maximum Revenue'],
    marker_color=['#ffff00','#000080', '#0000ff', '#e87d7d', '#008080',
                  '#00ffff', '#ffa500', '#c99c9c', '#7fffd4', '#8a2be2',
                 '#1f77b4',  '#ff7f0e','#2ca02c',  '#d62728', '#9467bd',  
                 '#8c564b', '#e377c2', '#7f7f7f',  '#bcbd22','#17becf',
                '#ff00ff', '#008000', '#00ff00', '#808000'],
    showlegend=False
))
fig8.add_trace(go.Bar(
    x=df_sec['Sector'],
    y=df_sec['Minimum Revenue'],
    text=df_sec['Minimum Revenue'],
    marker_color=['#ffff00','#000080', '#0000ff', '#e87d7d', '#008080',
                  '#00ffff', '#ffa500', '#c99c9c', '#7fffd4', '#8a2be2',
                 '#1f77b4',  '#ff7f0e','#2ca02c',  '#d62728', '#9467bd',  
                 '#8c564b', '#e377c2', '#7f7f7f',  '#bcbd22','#17becf',
                '#ff00ff', '#008000', '#00ff00', '#808000'],
    showlegend=False
))

fig8.add_trace(go.Scatter(
    x=df_sec['Sector'],
    y=df_sec['Average Revenue'],
    mode='lines+markers',
    name='Average Revenue'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig8.update_layout( xaxis_tickangle=-45,title='Maximum,Minimum & Average Revenue by each sector', 
                  xaxis_title='Sector',
                  yaxis_title='Revenue(Million Dollars)',
                  showlegend=True
                  )
fig8.update_traces(texttemplate='%{text:.2s}',textposition='auto',selector=dict(type='bar'))
fig8.show()

In [ ]:
#If anybody want to plot the chart with py express
'''fig8 = px.bar(df_sec,
              x='Sector',
              y='Maximum Revenue', 
              color='Sector', 
              title='Maximum Revenue by each sector',
              text='Maximum Revenue',
              labels={'Maximum Revenue':'Revenue(Million Dollars)'})
fig8.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig8.show()
fig9 = px.bar(df_sec,
              x='Sector',
              y='Minimum Revenue', 
              color='Sector',
              text='Minimum Revenue',
              title='Minimum Revenue by each sector',
              labels={'Minimum Revenue':'Revenue(Million Dollars)'})
fig9.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig9.show()'''

In [ ]:
df_ind = df.groupby('Industry', as_index=False)[['Minimum Revenue','Maximum Revenue']].mean()
df_ind.sort_values('Minimum Revenue',inplace=True, ascending=False)
df_ind['Average Revenue'] = df_ind[['Minimum Revenue','Maximum Revenue']].mean(axis=1)

In [ ]:
fig9 = go.Figure()
fig9.add_trace(go.Bar(
    x=df_ind['Industry'],
    y=df_ind['Maximum Revenue'],
    text=df_ind['Maximum Revenue'],
    name='Maximum Revenue'
))
fig9.add_trace(go.Bar(
    x=df_ind['Industry'],
    y=df_ind['Minimum Revenue'],
    text=df_ind['Minimum Revenue'],
    name='Minumum Revenue'

))
fig9.add_trace(go.Scatter(
    x=df_ind['Industry'],
    y=df_ind['Average Revenue'],
    mode='lines+markers',
    name='Average Revenue'
))
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig9.update_layout( xaxis_tickangle=-45,title='Maximum, Minimum & Average Revenue by each Industry', 
                  xaxis_title='Industry',
                  yaxis_title='Revenue(Million Dollars)',
                  )
fig9.update_traces(texttemplate='%{text:.2s}',textposition='auto',selector=dict(type='bar'))
fig9.show()

In [ ]:
df_job = df.groupby('Job_Title', as_index=False)[['Minimum Salary(in grand)','Maximum Salary(in grand)']].mean().sort_values(['Maximum Salary(in grand)','Minimum Salary(in grand)'], ascending=False).head(20)

In [ ]:
fig10 = go.Figure()
fig10.add_trace(go.Bar(
    y=df_job['Job_Title'],
    x=df_job['Minimum Salary(in grand)'],
    text=df_job['Minimum Salary(in grand)'],
    name='Minimum Revenue',
    orientation='h',
))
fig10.add_trace(go.Bar(
    y=df_job['Job_Title'],
    x=df_job['Maximum Salary(in grand)'],
    text=df_job['Maximum Salary(in grand)'],
    name='Maximum Revenue',
    orientation='h'

))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig10.update_layout(barmode='group', xaxis_tickangle=-45,title='Maximum and Minmum salaries of different jobs', 
                  xaxis_title='Salary in grand(K)',
                  yaxis_title='Job Title',
                  )
fig10.update_traces(texttemplate='%{text:.2s}',textposition='outside')
fig10.show()

In [ ]:
'''from wordcloud import WordCloud
# instantiate a word cloud object
job = df['Job Title']
wc = WordCloud(background_color='white').generate(str(' '.join(job)))

# display the cloud
fig = plt.figure()
fig.set_figwidth(14)
fig.set_figheight(18)

plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()'''

In [ ]:
fig10 = px.box(df['Rating'],points="all", title="Rating's")
fig10.show()

In [ ]:
df_found = df.value_counts('Founded')

In [ ]:
fig11 = px.bar(df_found,
               title='Number of comapny founded in differnet years',
                labels = {"Founded": "Year"})
fig11.show()


In [ ]:
usa_maps = df.groupby('Location State', as_index=False)['Job_Title'].count()

In [ ]:

fig11 = px.choropleth(data_frame=usa_maps,locations= ['AZ','CA','DE','FL','IL','NJ','NY','PA','TX'],  locationmode="USA-states", 
                      color = ['200', '600','16','128','200', '24','187','173','1000'],
                      labels={'color':'number of jobs', 'locations':'State'}, 
                      scope="usa") 
fig11.update_layout( title_text = 'Total No. of Jobs in different states',geo_scope='usa',showlegend=False)
fig11.show()


In [ ]:
usa_maps1 = df.groupby('Location State', as_index=False)[['Minimum Salary(in grand)', 'Maximum Salary(in grand)']].mean().sort_values(['Minimum Salary(in grand)', 'Maximum Salary(in grand)'],ascending=False)
usa_maps1['Average salary'] = usa_maps1[['Minimum Salary(in grand)', 'Maximum Salary(in grand)']].mean(axis=1)

In [ ]:
fig12 = px.choropleth(data_frame=usa_maps1,locations=['CA','IL', 'NY', 'DE', 'AZ', 'NJ', 'PA', 'TX', 'FL'] , locationmode="USA-states", 
                      color = ['127.61833333333334','102.58500000000001','99.38235294117646','112.96875','99.1625','97.10416666666666','100.73121387283237','89.821','61.9765625'],
                      labels={'color':'Average Salary(in grand"k")', 'locations':'State'}, 
                      scope="usa") 
fig12.update_layout( title_text = 'Average salary in different states',geo_scope='usa',showlegend=False)
fig12.show()

**Analyzing Job Description column**

In [ ]:
job = list(df['Job Description'])
job_des = [x.lower() for x in job]

In [ ]:
lang = ['javascript', 'html/css', 'java', 'powershell','python', 'sql', 'php', 'c#', 'c++','go']
database = ['mysql','microsoft sql', 'postgresql', 'sqlite', 'mongodb', 'redis', 'elasticsearch', 'oracle', 'mariadb', 'dynamodb']
platform = ['aws','linux','windows','docker','google cloud', 'macos', 'kubernetes','android','slack','azure']
webframe = ['asp.net','angular','django','express','flask','laravel','react.js','spring','vue.js','jquery']

In [ ]:
x = []
for a in lang:
    for y in job_des:
        if a in y:
            x.append(a)
df_lang = pd.DataFrame(data=x,columns=['Languages'])
df_lang= df_lang.value_counts().rename_axis('Language').reset_index(name='counts')



b = []
for a in webframe:
    for y in job_des:
        if a in y:
            b.append(a)
df_web = pd.DataFrame(data=b,columns=['webframe'])
df_web= df_web.value_counts().rename_axis('Webframe').reset_index(name='counts')


c = []
for a in database:
    for y in job_des:
        if a in y:
            c.append(a)
df_db = pd.DataFrame(data=c,columns=['database'])
df_db= df_db.value_counts().rename_axis('Database').reset_index(name='counts')


d = []
for a in platform:
    for y in job_des:
        if a in y:
            d.append(a)
df_pl= pd.DataFrame(data=d,columns=['platform'])
df_pl= df_pl.value_counts().rename_axis('Platform').reset_index(name='counts')


In [ ]:
fig_2 = make_subplots(rows=2, cols=2,
                      subplot_titles=("Top 10 languages ", 
                                      " Top 10 webframe ",
                                     " Top 10 database ",
                                     " Top 10 platform "))
fig_2.add_trace(go.Bar(
    x=df_lang['Language'],
    y=df_lang['counts'],
    name='',
    marker_color=['#ffff00','#000080', '#0000ff', '#e87d7d', '#008080',
                  '#00ffff', '#ffa500', '#c99c9c', '#7fffd4', '#8a2be2']
    ),
              row=1,col=1 )

fig_2.add_trace(go.Bar(
    x=df_web['Webframe'],
    y=df_web['counts'],
    name='',
    marker_color=['#ffff00','#000080', '#0000ff', '#e87d7d', '#008080',
                  '#00ffff', '#ffa500', '#c99c9c', '#7fffd4', '#8a2be2']
    ),
              row=1,col=2 )

fig_2.add_trace(go.Bar(
    x=df_db['Database'],
    y=df_db['counts'],
    marker_color=['#ffff00','#000080', '#0000ff', '#e87d7d', '#008080',
                  '#00ffff', '#ffa500', '#c99c9c', '#7fffd4', '#8a2be2'],
    name='',
    ),
              row=2,col=1 )

fig_2.add_trace(go.Bar(
    x=df_pl['Platform'],
    y=df_pl['counts'],
    marker_color=['#ffff00','#000080', '#0000ff', '#e87d7d', '#008080',
                  '#00ffff', '#ffa500', '#c99c9c', '#7fffd4', '#8a2be2',],
    name='',
    ),
              row=2,col=2 )

fig_2.update_layout(showlegend=False, 
                    plot_bgcolor='rgba(0,0,0,0)',
                    font=dict(family='Arial', 
                              size=12, 
                              color='black'))
fig_2.show()

In [ ]:
df_own = df['Type of ownership'].value_counts().rename_axis('Type of ownership').reset_index(name='counts')

In [ ]:
fig_4 = px.pie(df_own,
              names='Type of ownership',
              values='counts',
              title='Type of ownership',
              color_discrete_sequence=px.colors.sequential.RdBu)
fig_4.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')

fig_4.show()